In [1]:
# We initialize the libraries needed to work with twitter and create interactive graphics
from TwitterAPI import TwitterAPI
import json
import twitter
import time
import string
from nltk.corpus import words
import io
import numpy as np
from IPython.display import display
import plotly.tools as tls
import pandas as pd

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout

# We set the authorization needed to get tweets
CONSUMER_KEY = 'IZDwVjCfCnJjxIuggunDJxJom'
CONSUMER_SECRET = 'z2ZKVV0CnQpsShj48JOZbGedoRviDMrIGcb9eV74foKrmdGPEL'
ACCESS_TOKEN_KEY = '771636955041804288-if5bTd39lfygmHYUvu9Rasd8dto1UwA'
ACCESS_TOKEN_SECRET = '4VfUZ9Lhl2OQ3jGYmzJMMz0wlfuyzD4n0RBlM3P8QZMv9'


auth = twitter.oauth.OAuth(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth,retry=True)


        

ImportError: No module named TwitterAPI

In [2]:
# Creating a list with the names of the types of cuisines we are analyzing

cuisines=['irish cuisine', 'mexican cuisine', 'chinese cuisine', 'filipino cuisine', 'vietnamese cuisine', 'moroccan cuisine', 'brazilian cuisine', 'japanese cuisine', 'british cuisine', 'greek cuisine', 
          'indian cuisine', 'jamaican cuisine', 'french cuisine','spanish cuisine', 'russian cuisine', 'cajun_creole cuisine', 'thai cuisine', 'american cuisine', 'korean cuisine', 'italian cuisine']

 # Dictionary (keys: type of cuisines and values: tweets)
dict_tweets= {c:[] for c in cuisines}

In [ ]:
# For each cuisine we search for the tweets that has been posted in the last week. We search for them by the names of the list.

for cui in cuisines:
    SEARCH_TERM = cui
    count=200
    
    # We set the amount of tweets fetched to 200, however we have proved that we can only get 100 each time as max.
    results = twitter_api.search.tweets(q = SEARCH_TERM,count=count)
    statuses = results['statuses']

    print cui.upper()
    # Number of tweets per cuisine
    l=len(statuses)
    print ("We have %d statuses" %l)

    # We decide to iterate over 16 times to try to get 1600 tweets per cuisine, even though in some of them there will be less
    # because there are no more post with those characteristics
    # In this part, we get the id number of the last tweet of each iteration and insert it for doing the new request of tweets(here**)
    for i in range (0,15):
        for _ in range(5):
            try:
                next_results = results['search_metadata']['next_results']
            except KeyError, e: # No more results when next_results doesn't exist
                break

        # Create a dictionary from next_results, which has the following form:
        kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
        results = twitter_api.search.tweets(**kwargs ) # (here**)
        statuses += results['statuses']

        # Checking number of tweets in total
        l=len(statuses)
        print ("We have %d statuses" %l)


        tweets=[] # List of tweets per cuisine

        # We change the format of the tweets to json and add them into a list
        for status in statuses:
            j=json.dumps(status['text'], indent=1)
            tweets.append(j)

    # We insert each list of tweets per cuisine in the dictionary
    dict_tweets[cui].extend(tweets)
    


In [ ]:
# Save the tweets into a file:
with open('tweetsfile.json', 'w') as f:
    json.dump(dict_tweets, f)


In [15]:

# We check how many tweets are just the same one, this is because we want to make sure that we are not fetching the same
# tweets over and over again. We conclude to get some tweets that are the same and this could be dut to retweeting, because 
# there are not an excessive amount of each.
from collections import Counter

cnt=Counter()
# E.g: Checking number of the same tweets in Mexican cuisine
for tweet in dict_tweets['mexican cuisine']:
    cnt[tweet]+=1
print cnt

Counter({'"RT @sabores_CDMX: Chef @manolopitiona of @AguamielMx will bring the tradition of Mexican cuisine from #CDMX to #Vancouver. Enjoy #MexicoCit\\u2026"': 10, '"RT @e_chefsrecipes: Recipe contents page by cuisine type:\\n\\nhttps://t.co/eE0rwBnfUk\\n\\n#Malay #Thai #Indian #Mexican #Italian #Spanish #Scotti\\u2026"': 10, '"RT @SugarandRiceMag: Great essay by @LuvFoodAndMedia =&gt; Beyond Tex-Mex: Rediscovering Texas\\u2019 10,000-Year-Old Cuisine: https://t.co/PoG0T30z\\u2026"': 10, '"RT @LatinosHadleigh: Visit Latino\'s today and experience the finest #Spanish #Mexican and #Portuguese cuisine in Essex. #Tapas #Benfleet ht\\u2026"': 8, '"RT @one_emporio: Proud to work together . @MexlyFusion . Fusion Italiano &amp; Mexican cuisine . #comingsoon #restaurants #newopenings @NewOpen\\u2026"': 6, '"RT @JumiaFoodRW: Ever tried Mexican cuisine? No? Then you should try Meze Fresh\'s Quesadillas at only 3,000 Frw with a Free delivery! https\\u2026"': 6, '"RT @WildHibiscusCo: Hibis-kiss to

## --------------------------------CLEANING TWEETS--------------------------------------##

In [25]:
# Loading the content of the "tweetsfile"

with open('tweetsfile.json', 'r') as f:
    data = json.load(f)

In [26]:
# Testing if cleaning the punctuation in a particular tweet works.

data['brazilian cuisine'][0].replace("'", "")
data['brazilian cuisine'][0] = ''.join(ch for ch in data['brazilian cuisine'][0] if ch not in exclude)
data['brazilian cuisine'][0]

u'Stop by Wildfire amp do try our authentic Brazilian cuisine cooked with love by Chef Juarez G de Azevedou2026 httpstcop9pvvDngXA'

In [30]:
# Getting rid of the punctuation and words that don't exist in english for all the tweets.
# Tokenizing tweets and create a dictionary of tokens for each cuisine

exclude = set(string.punctuation)
data_clean={c:[] for c in cuisines} 
english_words = set(words.words())
data_tokens={c:[] for c in cuisines}


for cui in data_clean.keys():
    tweet_clean=[]
    tokens=[]
    print cui
    for tweet in data[cui]:
        tweet.replace("'", "")
        tweet = ''.join(ch for ch in tweet if ch not in exclude)
        tweet=[word for word in tweet.split() if word in english_words]
        tokens.extend(tweet)
    data_tokens[cui].extend(tokens)
print data_tokens['brazilian cuisine']

brazilian cuisine
mexican cuisine
irish cuisine
italian cuisine
french cuisine
chinese cuisine
vietnamese cuisine
jamaican cuisine
british cuisine
greek cuisine
filipino cuisine
japanese cuisine
korean cuisine
moroccan cuisine
spanish cuisine
cajun_creole cuisine
american cuisine
thai cuisine
indian cuisine
russian cuisine
[u'by', u'do', u'try', u'our', u'authentic', u'Brazilian', u'cuisine', u'with', u'love', u'by', u'G', u'de', u'eating', u'Brazilian', u'cuisine', u'at', u'de', u'ria', u'eating', u'Brazilian', u'cuisine', u'at', u'de', u'de', u'recipe', u'Brazilian', u'us', u'tonight', u'at', u'and', u'enjoy', u'all', u'your', u'favorite', u'Brazilian', u'us', u'tonight', u'at', u'and', u'enjoy', u'all', u'your', u'favorite', u'Brazilian', u'Os', u'da', u'os', u'eating', u'Brazilian', u'cuisine', u'at', u'Brazilian', u'Brazilian', u'and', u'for', u'or', u'at', u'to', u'whats', u'the', u'best', u'Brazilian', u'cuisine', u'recipe', u'book', u'available', u'in', u'English', u'with', u'o

In [31]:
# Save the dictionary of tokens in a file
with open('tokensfile.json', 'w') as f:
    json.dump(data_tokens, f)

## -------------------------------------- SENTIMENT ANALYSIS----------------------------------------------------- ##

In [3]:
# Open the file which contains the average happiness of some words.
f = io.open('Data_Set_S1.txt','r', encoding='utf-8')
data = f.read()
f.close()

In [4]:
# We load the file with the tokens
with open('tokensfile.json', 'r') as f:
    data1 = json.load(f)

In [5]:
# Create a dictionary in which we have as keys the words and as values their average sentiment.
sentiment_dict = {}
for line in (data.split('\n'))[1:-1]:
    try:
        sentiment_dict[line.split()[0]] = float(line.split()[2])
    except:
        print line
        raise

In [7]:
# Checking sentiment of word "happiness"
print sentiment_dict['happiness']

8.44


In [6]:
# Creating a funtion in which we will introduce the tokens for each cuisine and the dictionary of words with data of sentiment.
# We create a list in which we will have the average sentiment of each word, then we calculate the mean sentiment for that list
def sentiments(tokens, sentiment_dict):
    sent_list = [sentiment_dict[token] for token in tokens if token in sentiment_dict]
    
    if len(sent_list) == 0:
        print "The list has no sentimens... :_("
        return -1
    else:
        return float(np.mean(sent_list))


In [7]:
# Creating a dictionary with name of cuisines as keys and averagesentiment as values.
average_sent={c:'' for c in cuisines}

for cui in cuisines:
    sentiment=sentiments(data1[cui], sentiment_dict)
    average_sent[cui]=round(sentiment,2)
print average_sent

{'brazilian cuisine': 5.66, 'mexican cuisine': 5.6, 'irish cuisine': 5.68, 'italian cuisine': 5.58, 'french cuisine': 5.66, 'chinese cuisine': 5.74, 'vietnamese cuisine': 5.66, 'jamaican cuisine': 5.65, 'british cuisine': 5.45, 'greek cuisine': 5.6, 'filipino cuisine': 5.61, 'japanese cuisine': 5.62, 'korean cuisine': 5.57, 'moroccan cuisine': 5.65, 'spanish cuisine': 5.54, 'cajun_creole cuisine': 5.71, 'american cuisine': 5.0, 'thai cuisine': 5.7, 'indian cuisine': 5.68, 'russian cuisine': 5.68}


In [8]:
# Save the the value for the sentiment for each cuisine in a file
with open('sentimentfile.json', 'w') as f:
    json.dump(average_sent, f)

In [9]:
cocinas=[]
for key in average_sent.keys():
    cocinas.append(key[:-7])
print cocinas

['brazilian ', 'mexican ', 'irish ', 'italian ', 'french ', 'chinese ', 'vietnamese ', 'jamaican ', 'british ', 'greek ', 'filipino ', 'japanese ', 'korean ', 'moroccan ', 'spanish ', 'cajun_creole ', 'american ', 'thai ', 'indian ', 'russian ']


In [14]:
# Creating a chart in which we can see the average sentiment per type of cuisine
py.iplot({'data':[Bar(x=cocinas, y=average_sent.values())],
          'layout':Layout(barmode='stack', xaxis= {'tickangle': 40},yaxis={'range':[4.9,5.7]}, title='Sentiment per Cuisine')} ,
         filename='Sentiment per Cuisine')

In [15]:
# Creating a chart in which we can see the average sentiment per type of cuisine(excluding America due to its low sentiment 
# and the big difference with the rest of cuisines)
py.iplot({'data':[Bar(x=cocinas, y=average_sent.values())],
          'layout':Layout(barmode='stack', xaxis= {'tickangle': 40},yaxis={'range':[5.4,5.7]}, title='Sentiment per Cuisine')} ,
         filename='Sentiment per Cuisine')

In [16]:
# Sentiment average per cuisine located in the world map

# Loading the file with the names of the countries and codes
cod = pd.read_csv('country.csv')


data = [ dict(
        type = 'choropleth',
        locations = cod['CODE'], # Introducing code per country
        z = average_sent.values(), # Introducing average happinessof each country
        text = cod['COUNTRY'], # Introducing country names
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Grade of Happiness'),
      ) ]

layout = dict(
    title = 'Happiness av. per cuisine in the World',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )